In [ ]:
import h5py
import random
import numpy as np
from matplotlib.pyplot import imshow, show, subplot, figure, axis, plot, xlabel,ylabel,title,savefig
 ######################################################################################
######################################################################################
################################      Question 2       #####################################
######################################################################################
######################################################################################
def q2(): 
    filename = "data2.h5"
    with h5py.File(filename, "r") as f:
        # List all groups
        print("Keys: %s" % f.keys())
        testSet_Y = f[list(f.keys())[0]][:]
        testSet_X = f[list(f.keys())[1]][:]
        trainSet_Y = f[list(f.keys())[2]][:]
        trainSet_X = f[list(f.keys())[3]][:]
        valueOf_y = f[list(f.keys())[4]][:]
        valueOf_x = f[list(f.keys())[5]][:]
        words = f[list(f.keys())[6]][:]
        
    def vectorOfWord(x, maxInd = 250):
      """
      The vectorOfWord function converts a word index to a one-hot encoded vector. 
      The input x is an integer representing the index of a word in a vocabulary 
      of words. The one-hot encoded vector is a list of zeros with a single element 
      of 1 at the index x-1,which represents the position of the word in the
      vocabulary. The optional parameter maxInd specifies the size of the 
      one-hot encoded vector, and it is set to 250 by default.

      """
      out = np.zeros(maxInd)
      out[x-1] = 1
      return out

    def input_vector(data):
      """
      he input_vector function encodes a list of three-word phrases as a 
      list of one-hot encoded vectors.The input data is a 2D array where 
      each row represents a three-word phrase, with the first, second, 
      and third elements representing the indices of the words in the 
      vocabulary.

      """
      dataEncoded = []
      for row in data:
          word_1 = vectorOfWord(row[0])
          word_2 = vectorOfWord(row[1])
          word_3 = vectorOfWord(row[2])
          row = np.concatenate((word_1,word_2,word_3))
          row = row.reshape(1,len(row))
          dataEncoded.append(row)
      return dataEncoded

    def output_vector(data):
      """
      The function first initializes an empty list dataEncoded. It then iterates
      over each element in data, converts the word index to a one-hot encoded 
      vector using the vectorOfWord function, and adds it to the dataEncoded list. 
      Finally, the function returns the dataEncoded list.
      """
      dataEncoded = []
      for row in data:
          word = vectorOfWord(row)
          dataEncoded.append(word)
      return dataEncoded

    def init_coeff(D,P,data,mean=0,std=0.01):
      """
       To initialize the weights and biases of a neural network with a particular
       architecture. The weights and biases are initialized as random normal 
       variables with a given mean and standard deviation.
      """
      N = 200
      W0 = np.random.normal(mean,std, 750*D).reshape(750, D)
      W1 = np.random.normal(mean,std, D*P).reshape(D,P)
      W2 = np.random.normal(mean,std, P*250).reshape(P,250)
      b1 = np.random.normal(mean,std, N*P).reshape(N,P)
      b2 = np.random.normal(mean,std, N*250).reshape(N,250)
      
      Mul = [W0,W1,W2,b1,b2]
      
      return Mul

    def sigmoid(x):
        y = 1 / (1 + np.exp(-x))
        return y

    def sigmoid_backward(x):
      """
      This function can be used in the backpropagation algorithm to calculate 
      the gradient of the loss function with respect to the weights and biases 
      of the neural network. Backpropagation is an algorithm used to train 
      neural networks by updating the weights and biases to minimize the loss 
      function. The gradients calculated using backpropagation are used to update 
      the weights and biases using an optimization algorithm such as stochastic
      gradient descent.
      """
      d_sig = x*(1-x)
      return d_sig

    def softmax(x):
      """
      Softmax function, which is a generalization of the logistic function to 
      multiple classes. It is often used as the activation function for the 
      output layer of a neural network when the network is used for classification 
      tasks with more than two classes.
      """
      expx = np.exp(x - np.max(x))
      y = expx/np.sum(expx, axis=0)
      return y

    def cross_entrophy(y,y_pred):
      """
      The cross-entropy loss function, which is a commonly used loss function 
      for classification tasks. 
      """
      return (np.sum(- y * np.log(y_pred))/ y.shape[0])

    def forward(data, Mul):
      """
      The function you provided appears to implement the forward pass of a neural 
      network with a particular architecture. The forward pass is the process of
      computing the output of the neural network given an input data point.

      The function takes in two arguments: data and Mul. The variable data 
      represents the input data and has shape (batch size, number of features). 
      The variable Mul is a list containing the weights and biases of the neural 
      network. The function unpacks Mul into the variables w0, w1, w2, b1, and b2, 
      which represent the weights and biases of the first, second, and output 
      layers of the neural network, respectively.
      """ 
      
      w0,w1,w2,b1,b2 = Mul

      z0 = np.dot(data,w0) #first layer linear forward
      A0 = z0 #first layer without activation
      z1 = np.dot(A0,w1) + b1 #second layer linear forward
      A1 = sigmoid(z1) #second layer activation
      z2 =  np.dot(A1,w2) + b2 #output linear forward
      output = softmax(z2) #output layer activation

      return A0,A1,output

    def calculationOfCost(data,y,Mul):
      """
      The function takes in three arguments: data, y, and Mul. The variable data 
      represents the input data and has shape (batch size, number of features). 
      The variable y represents the true labels and has shape (batch size, number 
      of classes). The variable Mul is a list containing the weights and biases 
      of the neural network.

    """
      N = data.shape[0]
      W0,W1,W2,b1,b2 = Mul
      A0,A1,output = forward(data, Mul)
      d_sig = sigmoid_backward(A1)
  
      amountOfCost = cross_entrophy(y,output)

      d_w0 = np.dot(data.T,((np.dot(((np.dot((y-output),W2.T))*d_sig),W1.T))*A0))
      d_w1 = np.dot(A0.T,(np.dot((y-output),W2.T)*d_sig))
      d_w2 = np.dot(A1.T,(y-output))
      d_b1 = np.dot((y-output),W2.T)*d_sig
      d_b2 = y-output
      
      grads = [d_w0,d_w1,d_w2,d_b1,d_b2]
      
      return amountOfCost, grads

    def backwardPropagation(data,y,lr_rate,momentum, Mul, old_grads):
      """
      The function first calculates the cost and gradients of the neural network
      using the forward and backward propagation algorithms. It then updates the 
      weights and biases of the network using the gradients and the learning rate
      and momentum hyperparameters. The updated weights and biases are stored in Mul.
      Finally, the function returns the cost (amountOfCost), grads, and Mul 
      as a tuple.
      """
      
      amountOfCost, grads = calculationOfCost(data,y,Mul)
  
      Mul[0] -= lr_rate*grads[0]+old_grads[0]*momentum
      Mul[1] -= lr_rate*grads[1]+old_grads[1]*momentum
      Mul[2] -= lr_rate*grads[2]+old_grads[2]*momentum
      Mul[3] -= lr_rate*grads[3]+old_grads[3]*momentum
      Mul[4] -= lr_rate*grads[4]+old_grads[4]*momentum

      return amountOfCost, grads, Mul
        
    def train(trainSet_X,trainSet_Y,D,P,epoch,num_batch,lr_rate,momentum):
      """
      To iterate over a number of epochs, and within each epoch it divides the
      training data into batches and performs the training by calling the 
      backwardPropagation function with the current batch of data and the 
      current model coefficients. The function also appears to keep track of 
      the cost (or loss) at each epoch and return the final model coefficients,
      the list of costs, and the list of epochs.
      """
      amountOfCosts = []
      epochs = []
      data = input_vector(trainSet_X)
      data = np.squeeze(data,axis=1)
      label = output_vector(trainSet_Y)
      label = np.array(label)
      Mul = init_coeff(D,P,data,mean=0,std=0.01)
      momentum = 0.0000085
      amountOfCost, old_grads = calculationOfCost(data[:200],label[0:200],Mul)
      lr_rate = 0.0000015
      for i in range (epoch):
          epochs.append(i)
          for j in range (num_batch):
              data_batch = data[200*j:200*j+200]
              label_batch = label[200*j:200*j+200]
              amountOfCost, grads,Mul = backwardPropagation(data_batch,label_batch, lr_rate,momentum,Mul,old_grads)
          amountOfCosts.append(amountOfCost)
      j = 0
      for i in reversed (amountOfCosts):
          print("Epoch: {} --------------> Loss: {} ".format(j+1,i))
          j+=1
      return Mul,amountOfCosts[::-1],epochs

    def random_index(sample_size):
       """
       The function takes in a single input, sample_size, which specifies the 
       length of the list of random indices to generate. The function uses a 
       for loop to generate sample_size number of random indices using the 
       random.randint function and appends each one to a list. Finally, the 
       function returns the list of random indices.

       """
       random_index = []
       for i in range(sample_size):
          index = random.randint(0,46500)
          random_index.append(index)
       return random_index

    def choose_Sample(data,label,sample_size):
      """
      This is a function that selects a random sample of data from a larger dataset. 
      It looks like the function takes in three inputs:

      data, which is the larger dataset from which the sample will be drawn
      
      label, which appears to be a list of labels corresponding to the data points in data
      
      sample_size, which specifies the number of data points to include in the sample
      
      The function calls the random_index function to generate a list of sample_size 
      number of random indices, and then uses a for loop to select the corresponding 
      data points and labels from data and label using the generated indices. 
      Finally, the function returns the selected sample of data and 
      the corresponding labels.
      """
      sample = []
      labels = []
      sample_index = random_index(sample_size)
      for i in sample_index:
          sample.append(data[i])
          labels.append(label[i])
      return sample,labels

    def predict(words,output):
      """
      To be making predictions based on a list of output vectors and a list of 
      words.The function takes in two inputs:

      words, which is a list of words
      
      output, which is a list of output vectors
      
      The function uses a for loop to iterate over the list of output vectors
      and for each vector, it uses the argsort method to get the indices of the
      k largest values, where k is 10 in this case. The function then uses a 
      second for loop to iterate over the list of indices and selects the 
      corresponding words from the words list. The function appends the list 
      of selected words to a list called pred_rows and returns this list at the end.

      """
      pred_rows = []
      for i in range(len(output)):
          word_index = output[i].argsort()[-10:][::-1]
          pred_words = []
          for word in word_index: 
              pred_words.append(str(words[word].decode("utf-8")))
          pred_rows.append((pred_words))
      return pred_rows

    def print_preds(casual_example,test_label,pred_rows,words):
      """
      This functions prints the predictions that we get from the earlier functions
      """
      for i in range(len(casual_example)):
          tri = "sample trigram: "
          for j in range(len(casual_example[i])):
              tri+=str(words[casual_example[i][j]].decode("utf-8"))+" "
          tri += " ----> label: " + str(words[test_label[i]].decode("utf-8"))
          print(tri)
          print("Top 10 predictions: ",pred_rows[i])
          
    def plot_losses(losses,epochs,titles):
      """
      The function using matplotlib to create the plot. It sets the x-axis label
      to "epoch", the y-axis label to "loss", and the title to the input titles.
      It then uses the plot function to plot the losses versus the epochs.

      """
      xlabel("epoch")
      ylabel("loss")
      title(titles)
      plot(epochs,losses)
    ########################## Test with different D and P values ######################
    
    """epoch changed to 5 in order to save tim TA while running main idea is to show code is running
    (default is 30 if you want to try you can change by yourself 5th parameter)"""
    
    Mul,amountOfCosts,epochs = train(trainSet_X,trainSet_Y,32,256,5,1000,0.15,0.85)
    """Mul,amountOfCosts,epochs = train(valueOf_x,valueOf_y,32,256,30,232,0.15,0.85)"""
    plot_losses(amountOfCosts,epochs,"D=32 - P=256")
    
    """this part is commented in order to save time for TA to while running
    
    Mul,amountOfCosts,epochs = train(trainSet_X,trainSet_Y,16,128,30,1000,0.15,0.85)
    Mul,amountOfCosts,epochs = train(valueOf_x,valueOf_y,16,32,30,232,0.15,0.85)
    plot_losses(amountOfCosts,epochs,"D=16 - P=128")
    
    Mul,amountOfCosts,epochs = train(trainSet_X,trainSet_Y,8,64,30,1000,0.15,0.85)
    Mul,amountOfCosts,epochs = train(valueOf_x,valueOf_y,16,32,30,232,0.15,0.85)
    plot_losses(amountOfCosts,epochs,"D=8 - P=64")"""
    
    ######################## Make Predictions with Test Data ##############################
    casual_example,test_label = choose_Sample(testSet_X,testSet_Y,200)
    casual_example_vector = input_vector(casual_example)
    casual_example_vector = np.squeeze(casual_example_vector,axis=1)
    casual_example = casual_example[:5]
    test_label  = test_label[:5]
    
    _,_,output = forward(casual_example_vector, Mul)
    output = output[:5]
    
    pred_rows = predict(words,output)
    print_preds(casual_example,test_label,pred_rows,words)

